In [3]:
# STD
import codecs
from functools import reduce
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import time
import math
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [15]:
batch_size = 128
embedding_dims = 100
en_file = "data/train/train.en"
fr_file = "data/train/train.fr"
num_epochs = 30
learning_rate = 0.01
max_sentence_len = 50

In [29]:
# same code as from model.py
# copied here so i could make adjustments as i see fit

# if code is commented out its old code that is no longer used

# >>>>>>
# is new code
# <<<<<<

class ParallelCorpus(Dataset):
    """
    Class that contains a parallel corpus used for training IBM Model 1 and 2.
    """
   
    #  def __init__(self, source_path, target_path):
    
    # >>>>>>>>>>>>>> added additional inputs.
    def __init__(self, source_path, target_path, max_sentence_size, cuda=False):
    # <<<<<<<<<<<<<<
    
        # Enable a way to read multiple paths into one corpus if wanted
        source_paths = source_path if self.is_listlike(source_path) else (source_path, )
        target_paths = target_path if self.is_listlike(target_path) else (target_path, )

        self.source_sentences = self.read_all(source_paths)
        self.target_sentences = self.read_all(target_paths)
        
        # >>>>> Limit to sentences shorter then max_sentence_size 
        self.source_sentences = [sentence for sentence in self.source_sentences if len(sentence) <= max_sentence_size]
        # <<<<<
        
        self.size = len(self.source_sentences)
        
        # self.source_vocab = set()
        # self._source_vocab_size = None
        
        
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
    
        # getting additional vocab info
        self.source_vocab, self.source_w2i, self.source_i2w, self.source_vocab_size = self.set_vocab(self.source_sentences)  
        self.target_vocab, self.target_w2i, self.target_i2w, self.target_vocab_size = self.set_vocab(self.target_sentences)
        
        # convert word sentences to index sentences
        self.source_idx = self.set_index_sentences(self.source_sentences, self.source_w2i)
        self.target_idx = self.set_index_sentences(self.target_sentences, self.target_w2i)
        
        # getting the lengths of the sentences
        self.source_lengths = [len(s) for s in self.source_idx]
        self.target_lengths = [len(s) for s in self.target_idx]
        
        # pad the sentences
        self.source_padded = self.pad_sequences(self.source_idx, self.source_lengths, self.source_w2i['<pad>'])
        self.target_padded = self.pad_sequences(self.target_lengths, self.target_lengths, self.target_w2i['<pad>'])
        
        # turn into tensors
        if (cuda):
            self.source_tensor = torch.LongTensor(self.source_padded).cuda()
            self.target_tensor = torch.LongTensor(self.target_padded).cuda()
            self.source_lengths = torch.LongTensor(self.source_lengths).cuda()
            self.target_lengths = torch.LongTensor(self.target_lengths).cuda()
        else:
            self.source_tensor = torch.LongTensor(self.source_padded)
            self.target_tensor = torch.LongTensor(self.target_padded)
            self.source_lengths = torch.LongTensor(self.source_lengths)
            self.target_lengths = torch.LongTensor(self.target_lengths)
            
        # sort them by sentence size, decending.
        self.source_tensor, self.target_tensor, self.source_lengths, self.target_lengths = \
            self.sort_data(self.source_tensor, self.target_tensor, self.source_lengths, self.target_lengths)
        
    def set_vocab(self, sentences):
        vocab = ['<pad>', '<unk>']
        for line in sentences:
            for word in line:
                if word not in vocab:
                    vocab.append(word)
                    
        word2idx = {w: idx for (idx, w) in enumerate(vocab)}
        idx2word = {idx: w for (idx, w) in enumerate(vocab)}

        vocab_size = len(vocab)
        
        return vocab, word2idx, idx2word, vocab_size
        
    def set_index_sentences(self, sentences, w2i):
        sentence_indexs = []
        for sentence in sentences:
            sentence_indexs.append([w2i[word] for word in sentence])
        return sentence_indexs
    
    def pad_sequences(self, sentence_indexs, seq_lengths, pad):
        padding = np.full((len(sentence_indexs),max(seq_lengths)),pad) 
        for idx, (seq, seqlen) in enumerate(zip(sentence_indexs, seq_lengths)):
            padding[idx, :seqlen] = seq
        return padding
    
    def sort_data(self, source, target, source_lengths ,target_lengths):

        target_lengths, perm_idx = target_lengths.sort(0, descending=True)
        source_tensor = source[perm_idx]
        target_tensor = target[perm_idx]
        source_lengths = source_lengths[perm_idx]
        return source_tensor, target_tensor, source_lengths, target_lengths
    
    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

        

    def read_all(self, paths):
        """
        Read all corpus file at given paths and merge results.
        """
        def _combine_lists(a, b):
            a.extend(b)
            return a

        sentences_per_path = [self.read_corpus_file(path) for path in paths]
        return reduce(_combine_lists, sentences_per_path)

    @staticmethod
    def read_corpus_file(path, filter_characters=[]):
        sentences = []

        with codecs.open(path, "rb", "utf-8") as corpus:
            for line in corpus.readlines():
                tokens = [token for token in line.strip().split() if token not in filter_characters]
                sentences.append(tokens)

        return sentences

#     @property
#     def parallel_sentences(self):
#         return zip(self.source_sentences, self.target_sentences)

#     @property
#     def source_vocab_size(self):
#         if self._source_vocab_size is None:
#             self.source_vocab = {token for sentence in self.target_sentences for token in sentence}
#             self._source_vocab_size = len(self.source_vocab)
#         return self._source_vocab_size

#     def __iter__(self):
#         return (
#             (source_sentence, target_sentence)
#             for source_sentence, target_sentence in zip(self.source_sentences, self.target_sentences)
#         )

#     def __len__(self):
#         return self.size

#     def __getitem__(self, item):
#         return self.source_sentences[item], self.target_sentences[item]


    # >>>>>>>>>>>>>
    def __len__(self):
        return self.source_tensor.size(0) 

    def __getitem__(self, index):
        return self.source_tensor[index], self.target_tensor[index], self.source_lengths[index], self.target_lengths[index]
    # <<<<<<<<<<<<<<<
    
    
    @staticmethod
    def is_listlike(obj):
        return type(obj) in (tuple, list, set)


In [30]:
dataset = ParallelCorpus(fr_file, en_file, max_sentence_len, True)

In [31]:
data_loder = DataLoader(dataset, batch_size=batch_size)